Spam: 
Credit: 3741078
Breast: 262576

### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config_Spam.yaml') # 读取配置文件

In [ ]:
hparams.max_epochs

In [ ]:
hparams.pro_type_file is None

In [ ]:
from modelModule.model5 import VAE5
import torch
model = VAE5(dim=16, pro_types=pro_types)

### 得到数据集相关属性

In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
# data_pd = pd.read_csv('Letter_train.csv')
# col_names = list(data_pd)
# df[col_name].value_counts()
# pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset, ValidDataset
from torch.utils.data import DataLoader

In [ ]:
train_dataset = FlatDataset('/root/autodl-tmp/VAE_pytorchLightning/Spam/Spam_without_label.csv', 
                    pro_type_file='Spam/Spam_pro_type.pkl',
                    replace_dict_file='Spam/Spam数值置换表.pkl')


In [ ]:
valid_dataset = ValidDataset(miss_file='Spam/Spam模拟缺失_0_3.csv', 
                            complete_file='/root/autodl-tmp/VAE_pytorchLightning/Spam/Spam_without_label.csv',
                            pro_type_file='Spam/Spam_pro_type.pkl',
                            replace_dict_file='Spam/Spam数值置换表.pkl')

In [ ]:
valid_dataloader = DataLoader(valid_dataset, 
                             batch_size=10, 
                             collate_fn=valid_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
val_batch = None
for i in valid_dataloader:
    val_batch = i
    break

In [ ]:
val_batch['global_normal'][1]

In [ ]:
val_batch['portion_normal'][1]

In [ ]:
val_batch['miss_matrix'][1]

In [ ]:
train_dataloader = DataLoader(train_dataset, 
                             batch_size=10, 
                             collate_fn=train_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch['src_data']

In [ ]:
batch["global_normal"]

In [ ]:
batch["portion_normal"][0]

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
a[:,0].reshape(-1,1)

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
isinstance(model, torch.nn.Embedding)

In [ ]:
a = torch.randint(0,2,size=(5, 3))
b = torch.nn.Parameter(torch.randn(1, 4))
print(b)
print(a)

miss_matric = a.unsqueeze (-1).expand(5,3,4)
print(miss_matric)
miss_matric*b

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

In [ ]:
b.shape

### 模型5精度测定

In [ ]:
from dataModule.dataset1 import FlatDataset, ValidDataset
from modelModule.model5 import VAE5
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd
import pickle

In [ ]:
valid_dataset = ValidDataset(miss_file='Credit_Card/Credit模拟缺失_0_3.csv', 
                            complete_file='Credit_Card/Credit_Card_without_label.csv',
                            pro_type_file='Credit_Card/Credit_Card_pro_type.pkl',
                            replace_dict_file='Credit_Card/Credit_Card数值置换表.pkl')

valid_dataloader = DataLoader(valid_dataset, 
                             batch_size=256, 
                             collate_fn=valid_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
val_batch = None
for i in valid_dataloader:
    val_batch = i
    break

In [ ]:
## 模型参数载入
model = VAE5(dim=23, 
            pro_types=pickle.load(open('Credit_Card/Credit_Card_pro_type.pkl', 'rb')),
            replace_dict=pickle.load(open('Credit_Card/Credit_Card数值置换表.pkl', 'rb')))# nhead=3
checkpoint_path = '模型参数保存/Credit_model5_/version_3/checkpoints/epoch=1282-step=151394.ckpt'    


checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()

print('载入成功')

In [ ]:
global_normal, portion_normal = val_batch['global_normal'], val_batch['portion_normal']
M_matrix = val_batch['miss_matrix']

imputed_data, mu, log_var = model(portion_normal, M_matrix) # [batch, dim]

# imputed_data = imputed_data * global_max + global_min # 恢复原来的值
miss_data_MSE = torch.sum(((1-M_matrix) * global_normal - (1-M_matrix) * imputed_data)**2)

In [ ]:
miss_file = 'Credit_Card/Credit模拟缺失_0_3.csv'      # 缺失数据集
miss_data = pd.read_csv(miss_file)
_, output, portion_normal_data_batch = model.inference(miss_data)



In [14]:
def minmax_norm(src_data, pro_types):
    '''
    对数据进行按列进行最大-最小正则化(减去最小值除以最大值与最小值的差),使得每一个数据都处于[0,1]区间
    src_data: 需要被归一化的数据, DF
    pro_types: 为一个列表，里面元素为('normal', None),('discrete', 2),表示每一列的属性
    返回值：
         部分归一化数据集np, 完全归一化数据集np, 最小值np, 最大值np
    '''
    Min_Val = src_data.min().to_numpy()   
    Max_Val = src_data.max().to_numpy()   
    norm_data = src_data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))) # 将缺失数据进行归一化

    partial_norm_data = norm_data.copy()
    for i in range(norm_data.shape[1]):
        if pro_types[i][0] == 'discrete':
            partial_norm_data.iloc[:,i] = src_data.iloc[:,i]
    return partial_norm_data.to_numpy(), norm_data.to_numpy(), Min_Val, Max_Val

In [16]:
complete_data = pd.read_csv("Credit_Card/Credit_Card_without_label.csv")
_, norm_complete_data, _, _ = minmax_norm(complete_data, pickle.load(open('Credit_Card/Credit_Card_pro_type.pkl', 'rb')))
global_normal_batch = norm_complete_data[:256]

In [20]:
(((1-M_matrix) * global_normal_batch - (1-M_matrix) * output[:256])**2).sum() 

tensor(6.3913, dtype=torch.float64)

In [ ]:
imputed_data

In [ ]:
miss_file = 'Breast\Breast模拟缺失_0_3.csv'      # 缺失数据集
miss_data = pd.read_csv(miss_file)

c, _ = model.inference(miss_data)


In [ ]:
c.shape

In [ ]:
# ## 模型参数载入
# model = VAE5(dim=23, pro_types=pickle.load(open('Credit_Card\Credit_Card_pro_type.pkl', 'rb'))) # nhead=3
# checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=31-step=3008.ckpt'    
# data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
# train_csv = r'Credit_Card_model_train.csv'
# test_csv = r'Credit_Card_model_test.csv'

# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
# model.eval()
# ## 载入训练集的均值(最小)方差(最大)
# train_dataset = FlatDataset(train_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# model.get_global_min_max(dataset=train_dataset)
# print('载入成功')

In [ ]:

# test_dataset = FlatDataset(test_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# data_size = 10
# src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
# normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
# portion_normal_data = np.stack([test_dataset[index]['portion_normal_data'] for index in range(data_size)], axis=0)
# _, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
# input = portion_normal_data.copy()
# input[miss_matrix == 0] = np.nan

In [ ]:
# imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [ ]:
# print(normal_data[0])
# print(output[0])


In [ ]:
imputed_data[1]

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))